<# Project 2
## Introduction
The Poisson equation appears in many physical applications: electrostatics, fluid mechanics, gravitational field, heat conduction, etc. Let us consider the Poisson problem in two space dimensions $x$ and $y$. It takes the form

$$
-\Delta u=f(x, y), \quad(x, y) \in \Omega, \quad \Delta=\frac{\partial^2}{\partial x^2}+\frac{\partial^2}{\partial y^2}
$$

where $u=u(x, y)$ is the function we are seeking, $f=f(x, y)$ is a source term, and $\Omega$ is the computational domain. In this project you will design a Poisson solver over a square domain

$$
\Omega=\{(x, y), 0 \leq x \leq 1,0 \leq y \leq 1\}
$$

and analyze its numerical properties. We propose to discretize the Poisson equation by a classical finite difference scheme, and to investigate the numerical performance of various linear system solvers. In the first part you will focus on finite difference discretization in two-dimensions and the verification of the code. In the second part you will test different direct and iterative linear system solvers, and try to understand their performance.
You should write a small project report that shows your results, and summarize your findings. For the evaluation I will pay attention to the quality of the interpretations, how you apply the algorithms and the quality of the numerical illustrations. It does not need to be long: I will prefer a short summary that shows that you have understood and analyzed the methods.

## 1 Finite difference discretization
In this part we propose to i) find a discrete version of the Poisson problem, ii) implement and validate the solver.

## 1.1 Designing the solver

You may follow the steps below
1. Generate a rectangular grid of $\left(N_x-1\right) \times\left(N_y-1\right)$ points, which will represent the discretization of the interior domain of $\Omega$. Each point ( $x_i, y_j$ ) of the grid has an index $i \in\left\{1, \cdots, N_x-1\right\}$ and $j \in\left\{1, \cdots, N_y-1\right\}$. You can use the numpy. meshgrid function.

In [3]:
# Step 1: Generate the grid
import numpy as np
import matplotlib.pyplot as plt

Nx = 10
Ny = 10
x = np.linspace(0, 1, Nx)  # Create an array of Nx points between 0 and 1
y = np.linspace(0, 1, Ny)  # Create an array of Ny points between 0 and 1
X, Y = np.meshgrid(x, y)   # Generate the meshgrid from x and y


2. Recall the finite difference formula for the second order derivative in 1D and its approximation order. Apply the formula to the $x$ and $y$ directions.


In [4]:
# Define the grid size and spacing
hx = 1 / (Nx - 1)  # Grid spacing in the x direction
hy = 1 / (Ny - 1)  # Grid spacing in the y direction

# Create a meshgrid for x and y
x = np.linspace(0, 1, Nx)  # x-coordinates
y = np.linspace(0, 1, Ny)  # y-coordinates
X, Y = np.meshgrid(x, y)  # Create a 2D grid

# Define the function u(x, y) = sin^2(pi x) * sin^2(pi y)
u = np.sin(np.pi * X)**2 * np.sin(np.pi * Y)**2  # Example function

# Initialize arrays for second derivatives
d2u_dx2 = np.zeros((Nx, Ny))
d2u_dy2 = np.zeros((Nx, Ny))

# Compute second derivatives for interior points
for i in range(1, Nx - 1):
    for j in range(1, Ny - 1):
        # d^2u/dx^2 at (i, j):
        d2u_dx2[i, j] = (u[i+1, j] - 2*u[i, j] + u[i-1, j]) / hx**2
        
        # d^2u/dy^2 at (i, j):
        d2u_dy2[i, j] = (u[i, j+1] - 2*u[i, j] + u[i, j-1]) / hy**2

# Now d2u_dx2 and d2u_dy2 contain the second derivatives for the interior points
#d2u_dx2, d2u_dy2 

3. Combine the two partial derivatives and write an explicit form of the discrete Laplacian in terms of the grid points. Use the notation shorthand $u_{i j}=u\left(x_i, y_j\right)$ for the unknowns. What simplification do you obtain with an uniform grid?


## The Discrete Laplacian

The Laplacian operator in 2D is given by:

1. Second-order derivative in $x$ :

$$
\left.\frac{\partial^2 u}{\partial x^2}\right|_{(i, j)} \approx \frac{u_{i+1, j}-2 u_{i, j}+u_{i-1, j}}{\Delta x^2}
$$

2. Second-order derivative in $y$ :

$$
\left.\frac{\partial^2 u}{\partial y^2}\right|_{(i, j)} \approx \frac{u_{i, j+1}-2 u_{i, j}+u_{i, j-1}}{\Delta y^2}
$$


Combining the Approximations
Combining the $x$ - and $y$-direction derivatives, the discrete Laplacian becomes:

$$
\Delta u_{i, j} \approx \frac{u_{i+1, j}-2 u_{i, j}+u_{i-1, j}}{\Delta x^2}+\frac{u_{i, j+1}-2 u_{i, j}+u_{i, j-1}}{\Delta y^2}
$$


In [5]:
import sympy as sp
import plotly.graph_objects as go

def finite_difference_second_derivatives(u, hx, hy):
    """
    Compute the second-order derivatives of a 2D array u using finite difference approximations.

    Parameters:
    u : 2D numpy array
        The values of the function u at grid points.
    hx : float
        The grid spacing in the x direction (Δx).
    hy : float
        The grid spacing in the y direction (Δy).

    Returns:
    d2u_dx2 : 2D numpy array
        The second derivative of u with respect to x.
    d2u_dy2 : 2D numpy array
        The second derivative of u with respect to y.
    """
    # Initialize the second derivative arrays
    d2u_dx2 = np.zeros_like(u)
    d2u_dy2 = np.zeros_like(u)

    # Compute the second derivative with respect to x
    d2u_dx2[1:-1, 1:-1] = (u[2:, 1:-1] - 2 * u[1:-1, 1:-1] + u[:-2, 1:-1]) / hx**2

    # Compute the second derivative with respect to y
    d2u_dy2[1:-1, 1:-1] = (u[1:-1, 2:] - 2 * u[1:-1, 1:-1] + u[1:-1, :-2]) / hy**2

    return d2u_dx2, d2u_dy2



## Simplification with a Uniform Grid
For a uniform grid, the grid spacing in both directions is equal $(\Delta x=\Delta y=h)$. This simplifies the Laplacian to:

$$
-\Delta u_{i, j}=\frac{-u_{i+1, j}+2 u_{i, j}-u_{i-1, j}}{h^2}+\frac{-u_{i, j+1}+2 u_{i, j}-u_{i, j-1}}{h^2} .
$$


Factoring out $h^2$, we obtain:

$$
-\Delta u_{i, j}=\frac{-u_{i+1, j}-u_{i-1, j}-u_{i, j+1}-u_{i, j-1}+4 u_{i, j}}{h^2}
$$


In [6]:
import sympy as sp
import plotly.graph_objects as go

# Create a meshgrid for x and y
x = np.linspace(0, 1, Nx)  # x-coordinates
y = np.linspace(0, 1, Ny)  # y-coordinates
X, Y = np.meshgrid(x, y)  # Create a 2D grid

# Define the function u(x, y) = sin^2(pi x) * sin^2(pi y)
u = np.sin(np.pi * X)**2 * np.sin(np.pi * Y)**2  # Example function

# Compute the first derivatives using np.gradient
du_dx = np.gradient(u, hx, axis=0)  # Gradient in x direction
du_dy = np.gradient(u, hy, axis=1)  # Gradient in y direction

# Compute the second derivatives
d2u_dx2 = np.gradient(du_dx, hx, axis=0)  # Second derivative in x direction
d2u_dy2 = np.gradient(du_dy, hy, axis=1)  # Second derivative in y direction

# Compute the Laplacian
laplacian = d2u_dx2 + d2u_dy2

# Compute the exact Laplacian for u(x, y) = sin^2(pi x) * sin^2(pi y)
# The exact Laplacian can be derived or computed separately; for this example, we will use a known result.
# The exact Laplacian of sin^2(pi x) * sin^2(pi y) is:
exact_laplacian = 2 * (2 * np.pi**2) * np.sin(np.pi * X)**2 * np.sin(np.pi * Y)**2 - 4 * np.pi**2 * np.cos(np.pi * X)**2 * np.cos(np.pi * Y)**2

# Compute the error
error = np.max(np.abs(laplacian - exact_laplacian))

# Pretty print the matrices using sympy
sp.init_printing()  # Initialize pretty printing
sympy_laplacian = sp.Matrix(laplacian)
sympy_exact_laplacian = sp.Matrix(exact_laplacian)

print("Numerical Laplacian:")
sp.pprint(sympy_laplacian)

print("\nExact Laplacian:")
sp.pprint(sympy_exact_laplacian)

# Print the maximum absolute error
print(f"\nMaximum absolute error: {error}")

# Plot results in 3D
fig1 = go.Figure(data=[go.Surface(z=laplacian, x=X, y=Y, colorscale='Viridis', showscale=True)])
fig1.update_layout(
    title='3D Surface of Numerical Laplacian',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Value',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    ),
    template='plotly_white'
)
fig1.show()

fig2 = go.Figure(data=[go.Surface(z=exact_laplacian, x=X, y=Y, colorscale='Viridis', showscale=True)])
fig2.update_layout(
    title='3D Surface of Exact Laplacian',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Value',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    ),
    template='plotly_white'
)
fig2.show()

Numerical Laplacian:
⎡         0            0.849074355273444  2.99900609385099   5.44381826142245 
⎢                                                                             
⎢ 0.849074355273444    1.89061824129193   3.67882558433499   5.08209386588796 
⎢                                                                             
⎢  2.99900609385099    3.67882558433499   2.40118286698649   -1.2776427173485 
⎢                                                                             
⎢  5.44381826142245    5.08209386588796   -1.2776427173485   -12.5502183016835
⎢                                                                             
⎢  7.03955607372047    5.99801218770198   -3.67882558433499  -19.9078694703535
⎢                                                                             
⎢  7.03955607372047    5.99801218770198   -3.67882558433499  -19.9078694703535
⎢                                                                             
⎢  5.44381826142245    5.082093

### Extended Interpretation of the Discrete Laplacian

The discrete Laplacian derived above is a **central difference approximation** to the continuous Laplacian operator 
$$
\Delta u = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}
$$
This approximation is commonly used in **finite difference methods** to solve partial differential equations (PDEs) numerically. Below, we provide a detailed interpretation of its components and applications:

---

#### 1. **Structure of the Discrete Laplacian**
- The discrete Laplacian uses a **5-point stencil**, which means the value of $\Delta u_{i,j}$ at the grid point $(i,j)$ is computed using the function values at five neighboring points:
  - $u_{i+1,j}$ (right neighbor),
  - $u_{i-1,j}$ (left neighbor),
  - $u_{i,j+1}$ (top neighbor),
  - $u_{i,j-1}$ (bottom neighbor),
  - and $u_{i,j}$ (the central point itself).
- These values are combined linearly with appropriate weights, such that the central point $u_{i,j}$ is penalized by $-4$ times its value, while each neighbor contributes $+1$.

This structure reflects how the value of a function at one grid point is influenced by its immediate surroundings, capturing the local curvature of the function.

---

#### 2. **Physical Interpretation**
The Laplacian operator represents the divergence of the gradient (or the flux) of a function. It measures how the value of a function at a point differs from its surroundings. Physically, this operator is associated with diffusion, heat flow, and wave propagation:

- In **heat transfer** problems, the Laplacian models the rate at which heat spreads out from a point.
- In **fluid dynamics**, it describes the diffusion of substances or momentum within a fluid.
- In **electromagnetism**, the Laplacian appears in Poisson's equation for electrostatics, modeling how charges distribute in space.

The discrete Laplacian provides a numerical approximation to this concept, enabling the simulation of such physical processes on a computational grid.

---

#### 3. **Simplification from a Uniform Grid**
The assumption of a uniform grid $h = \Delta x = \Delta y$ simplifies the numerical representation:
- All grid cells are square and equally spaced, leading to symmetry in the discrete Laplacian.
- The factor $\frac{1}{h^2}$ accounts for the uniform spacing and ensures dimensional consistency.
- This uniformity avoids the need for additional weighting or adjustments, as would be required for non-uniform grids.

The symmetry of the uniform grid also ensures that the discrete Laplacian preserves the properties of the continuous Laplacian, such as consistency and stability under smooth changes.

---

#### 4. **Error and Accuracy**
The discrete Laplacian introduces a **truncation error** due to the finite difference approximation. Specifically:
- The central difference approximation for the second derivative is second-order accurate, meaning the error decreases as $O(h^2)$ when the grid spacing $h$ becomes smaller.
- For sufficiently small $h$, the discrete Laplacian closely approximates the continuous Laplacian, making it a reliable tool for numerical simulations.

---

#### 5. **Applications**
The discrete Laplacian is a cornerstone of numerical methods for solving PDEs. Some key applications include:

1. **Heat Equation (Diffusion)**
   - The Laplacian models the diffusion of heat in a material. For example, in the equation:
   $$
   \frac{\partial u}{\partial t} = \alpha \Delta u,
   $$
   the discrete Laplacian is used to compute the spatial component, while time derivatives are handled with time-stepping schemes.

2. **Poisson's Equation**
   - In problems involving electrostatics or gravitational potential, the discrete Laplacian is used to approximate:
   $$
   \Delta u = f,
   $$
   where $f$ is a source term.

3. **Wave Equation**
   - In wave propagation, the Laplacian is part of the second-order spatial derivative:
   $$
   \frac{\partial^2 u}{\partial t^2} = c^2 \Delta u,
   $$
   where $c$ is the wave speed.


---

#### 6. **Numerical Implementation**
In practice, the discrete Laplacian is implemented as a **linear operator** on a grid. This makes it suitable for efficient computation using matrix representations:
- Each grid point corresponds to a row or column in a matrix.
- The Laplacian is represented as a sparse matrix with most entries being zero, except for the coefficients associated with the 5-point stencil.

This structure is crucial for solving large systems of equations efficiently using numerical solvers like iterative methods (e.g., conjugate gradient, Jacobi, Gauss-Seidel).

---

#### 7. **Limitations**
While the discrete Laplacian is highly useful, it has some limitations:
- **Boundary conditions**: Special care is needed to handle the boundaries of the grid (e.g., Dirichlet or Neumann conditions).
- **Non-uniform grids**: For irregularly spaced grids, the discrete Laplacian becomes more complex and requires alternative formulations.
- **Higher dimensions**: In 3D or higher dimensions, the stencil expands to include additional neighbors, increasing computational cost.

---



## 

## Laplacian manually implemented

In [7]:
def compute_laplacian(u, hx, hy):
    """
    Compute the Laplacian of a 2D array u using finite difference approximations.

    Parameters:
    u : 2D numpy array
        The values of the function u at grid points.
    hx : float
        The grid spacing in the x direction (Δx).
    hy : float
        The grid spacing in the y direction (Δy).

    Returns:
    laplacian : 2D numpy array
        The Laplacian of u.
    """
    laplacian = np.zeros_like(u)

    # Compute the Laplacian using finite difference
    laplacian[1:-1, 1:-1] = (
        (u[2:, 1:-1] - 2 * u[1:-1, 1:-1] + u[:-2, 1:-1]) / hx**2 +
        (u[1:-1, 2:] - 2 * u[1:-1, 1:-1] + u[1:-1, :-2]) / hy**2
    )

    return laplacian




## Compute the Laplacian with numpy

In [8]:
def finite_difference_2d(u, hx, hy):
    # Compute the gradient
    grad_u = np.gradient(u, hx, hy)
    
    # Second derivative in x direction
    d2u_dx2 = np.gradient(grad_u[0], hx, axis=0)  # Gradient in x direction
    # Second derivative in y direction
    d2u_dy2 = np.gradient(grad_u[1], hy, axis=1)  # Gradient in y direction
    
    return d2u_dx2, d2u_dy2

def compute_laplacian(u, hx, hy):
    # Compute the second derivatives
    d2u_dx2, d2u_dy2 = finite_difference_2d(u, hx, hy)
    # Compute the Laplacian
    laplacian = d2u_dx2 + d2u_dy2
    return laplacian


# Example usage
hx = 1 / (Nx - 1)
hy = 1 / (Ny - 1)

# Test 1: Zero grid
u1 = np.zeros((Nx, Ny))  # Example grid
laplacian_1 = compute_laplacian(u1, hx, hy)

# Test 2: Sinusoidal grid
u2 = np.sin(np.pi * np.linspace(0, 1, Nx)[:, None]) * np.sin(np.pi * np.linspace(0, 1, Ny))  # 2D sinusoidal function
laplacian_2 = compute_laplacian(u2, hx, hy)

# Test 3: Linear grid
u3 = np.linspace(0, 1, Nx)[:, None] * np.linspace(0, 1, Ny)  # 2D linear function
laplacian_3 = compute_laplacian(u3, hx, hy)

# Function to create 3D surface plots
def plot_3d_surface(X, Y, Z, title):
    fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='Viridis')])
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Laplacian Value',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
        ),
        template='plotly_white'
    )
    fig.show()

# Create meshgrid for plotting
X, Y = np.meshgrid(np.linspace(0, 1, Nx), np.linspace(0, 1, Ny))

# Plot results for Laplacian of Test 1
plot_3d_surface(X, Y, laplacian_1, '3D Surface of Laplacian (Zero Grid)')

# Plot results for Laplacian of Test 2
plot_3d_surface(X, Y, laplacian_2, '3D Surface of Laplacian (Sinusoidal Grid)')

# Plot results for Laplacian of Test 3
plot_3d_surface(X, Y, laplacian_3, '3D Surface of Laplacian (Linear Grid)')

4. Write the discrete Laplacian in a matrix form, and express the discrete Poisson equation as a linear system $\mathbb{A} \mathbf{u}=\mathbf{f}$ of size $\left(N_x-1\right) \times\left(N_y-1\right)$. To do so you need to order the unknowns to get a global unknown vector. For example, you can use the column ordering $\mathbf{u}=\mathbf{u}_j, j=\left\{1, N_y-1\right\}$, where each $\mathbf{u}_j$ is a vector of size $\left(N_x-1\right)$ such that

$$
\mathbf{u}_j=\left(u_{0 j}, u_{1 j}, \cdots, u_{N_x-1, j}\right)^T
$$




In [9]:
# Define the function u(x, y) = sin^2(pi x) * sin^2(pi y)
u = np.sin(np.pi * X)**2 * np.sin(np.pi * Y)**2  # Example function

# Compute the first derivatives using np.gradient
du_dx = np.gradient(u, hx, axis=0)  # Gradient in x direction
du_dy = np.gradient(u, hy, axis=1)  # Gradient in y direction

# Compute the second derivatives
d2u_dx2 = np.gradient(du_dx, hx, axis=0)  # Second derivative in x direction
d2u_dy2 = np.gradient(du_dy, hy, axis=1)  # Second derivative in y direction

# Compute the Laplacian
laplacian = d2u_dx2 + d2u_dy2

# Compute the exact Laplacian for u(x, y) = sin^2(pi x) * sin^2(pi y)
exact_laplacian = 2 * (2 * np.pi**2) * np.sin(np.pi * X)**2 * np.sin(np.pi * Y)**2 - 4 * np.pi**2 * np.cos(np.pi * X)**2 * np.cos(np.pi * Y)**2

# Compute the error
error = np.max(np.abs(laplacian - exact_laplacian))

# Print the results
print("Numerical Laplacian:")
print(laplacian)

print("\nExact Laplacian:")
print(exact_laplacian)

# Print the maximum absolute error
print(f"\nMaximum absolute error: {error}")

Numerical Laplacian:
[[ 0.00000000e+00  8.49074355e-01  2.99900609e+00  5.44381826e+00
   7.03955607e+00  7.03955607e+00  5.44381826e+00  2.99900609e+00
   8.49074355e-01  1.08858929e-31]
 [ 8.49074355e-01  1.89061824e+00  3.67882558e+00  5.08209387e+00
   5.99801219e+00  5.99801219e+00  5.08209387e+00  3.67882558e+00
   1.89061824e+00  8.49074355e-01]
 [ 2.99900609e+00  3.67882558e+00  2.40118287e+00 -1.27764272e+00
  -3.67882558e+00 -3.67882558e+00 -1.27764272e+00  2.40118287e+00
   3.67882558e+00  2.99900609e+00]
 [ 5.44381826e+00  5.08209387e+00 -1.27764272e+00 -1.25502183e+01
  -1.99078695e+01 -1.99078695e+01 -1.25502183e+01 -1.27764272e+00
   5.08209387e+00  5.44381826e+00]
 [ 7.03955607e+00  5.99801219e+00 -3.67882558e+00 -1.99078695e+01
  -3.05006256e+01 -3.05006256e+01 -1.99078695e+01 -3.67882558e+00
   5.99801219e+00  7.03955607e+00]
 [ 7.03955607e+00  5.99801219e+00 -3.67882558e+00 -1.99078695e+01
  -3.05006256e+01 -3.05006256e+01 -1.99078695e+01 -3.67882558e+00
   5.9980121

For example, you can use the column ordering $\mathbf{u}=\mathbf{u}_j, j=\left\{1, N_y-1\right\}$, where each $\mathbf{u}_j$ is a vector of size $\left(N_x-1\right)$ such that

$$
\mathbf{u}_j=\left(u_{0 j}, u_{1 j}, \cdots, u_{N_x-1, j}\right)^T
$$


Up to now we have not considered the boundary points. In any PDE solver we need to specify boundary conditions. Let us assume we use a homogeneous Dirichlet boundary condition everywhere, that is

$$
u(x, 1)=u(x, 0)=u(1, y)=u(0, y)=0, \quad(x, y) \in \partial \Omega .
$$


Show that the boundary condition can be incorporated into the linear system as

$$
\mathbb{A} \mathbf{u}=\mathbf{b}+\mathbf{f}
$$

where you will specify the vector $\mathbf{b}$, of size $\left(N_y-1\right)^2$. How would you extend the procedure for a non-zero boundary condition?

In [10]:
from scipy.sparse import diags

def create_poisson_system(Nx, Ny):
    # Step 1: Generate the grid
    hx = 1 / (Nx - 1)  # Grid spacing in the x direction
    hy = 1 / (Ny - 1)  # Grid spacing in the y direction

    # Step 2: Define the discrete Laplacian matrix A
    diagonals = [-4 * np.ones((Nx - 1) * (Ny - 1)), 
                 np.ones((Nx - 1) * (Ny - 1) - 1), 
                 np.ones((Nx - 1) * (Ny - 1) - 1), 
                 np.ones((Nx - 1) * (Ny - 1) - (Nx - 1)), 
                 np.ones((Nx - 1) * (Ny - 1) - (Nx - 1))]
    
    # Create the sparse matrix A
    A = diags(diagonals, [0, -1, 1, -(Nx - 1), (Nx - 1)], 
              shape=((Nx - 1) * (Ny - 1), (Nx - 1) * (Ny - 1))).tocsc() #Convert this array/matrix to Compressed Sparse Column format



    # Step 3: Define the right-hand side vector f
    f = np.zeros((Nx - 1) * (Ny - 1))  # Initialize f

    # Step 4: Define the boundary condition vector b
    b = np.zeros((Ny - 1) ** 2)  # Initialize b for the boundary conditions

    return A, f, b

# Example usage
Nx, Ny = 10, 10  # Define grid size
A, f, b = create_poisson_system(Nx, Ny)

# Display the results
print("Sparse Matrix A:\n", A.toarray())
print("Right-hand side vector f:\n", f)
print("Boundary condition vector b:\n", b)

Sparse Matrix A:
 [[-4.  1.  0. ...  0.  0.  0.]
 [ 1. -4.  1. ...  0.  0.  0.]
 [ 0.  1. -4. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -4.  1.  0.]
 [ 0.  0.  0. ...  1. -4.  1.]
 [ 0.  0.  0. ...  0.  1. -4.]]
Right-hand side vector f:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Boundary condition vector b:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## 1.2 Validation of the implementation

To validate the implementation we need to define an exact solution. To do so let us take

$$
u_{\mathrm{ex}}(x, y)=\sin ^2(\pi x) \sin ^2(\pi y)
$$


Find by hand the corresponding right-hand-side $f(x, y)$ for this solution. Is the exact solution consistent with the boundary conditions?



In [11]:
# Define the exact solution
def exact_solution(X, Y):
    return np.sin(np.pi * X) ** 2 * np.sin(np.pi * Y) ** 2

# Compute the right-hand side f(x, y) for the exact solution
def compute_rhs(X, Y):
    return -np.pi ** 2 * (2 * np.sin(np.pi * X) ** 2 * np.sin(np.pi * Y) ** 2 +
                          2 * np.sin(np.pi * X) ** 2 * np.cos(np.pi * Y) ** 2 +
                          2 * np.sin(np.pi * Y) ** 2 * np.cos(np.pi * X) ** 2)

# Check boundary conditions
def check_boundary_conditions(Nx, Ny):
    x = np.linspace(0, 1, Nx)
    y = np.linspace(0, 1, Ny)
    X, Y = np.meshgrid(x, y)
    
    # Evaluate the exact solution at the boundaries
    boundary_values = exact_solution(X, Y)
    
    # Check if the boundary values are zero
    return np.allclose(boundary_values[0, :], 0) and np.allclose(boundary_values[-1, :], 0) and \
           np.allclose(boundary_values[:, 0], 0) and np.allclose(boundary_values[:, -1], 0)

# Example usage
Nx, Ny = 10, 10  # Define grid size
b_conditions = check_boundary_conditions(Nx, Ny)
print(f"Boundary conditions satisfied: {b_conditions}")

Boundary conditions satisfied: True


Given a number of grid points (start with a small number), call a linear system solver and compute the relative error in the maximum norm in the computational domain $\Omega$. Show a convergence plot in log-log scale with respect to the step size. What is the expected convergence rate? Do you observe any difficulty in terms of computational time?

In [26]:
from scipy.sparse.linalg import spsolve
import plotly.graph_objects as go
import time

start_time = time.time()

# Define the exact solution
def exact_solution(X, Y):
    return np.sin(np.pi * X) ** 2 * np.sin(np.pi * Y) ** 2

# Define the right-hand side f(x, y)
def compute_rhs(X, Y):
    return -np.pi ** 2 * (2 * np.sin(np.pi * X) ** 2 * np.sin(np.pi * Y) ** 2 +
                          2 * np.sin(np.pi * X) ** 2 * np.cos(np.pi * Y) ** 2 +
                          2 * np.sin(np.pi * Y) ** 2 * np.cos(np.pi * X) ** 2)

# Validation function
def validate_solver(Nx, Ny):
    # Create the Poisson system
    A, f, b = create_poisson_system(Nx, Ny)
    
    # Solve the linear system
    u = spsolve(A, f + b)  # Incorporate boundary conditions

    # Create grid for exact solution
    x = np.linspace(0, 1, Nx)
    y = np.linspace(0, 1, Ny)
    X, Y = np.meshgrid(x, y)

    # Compute exact solution and error
    u_ex = exact_solution(X, Y).flatten()[:(Nx - 1) * (Ny - 1)]  # Ensure u_ex matches the size of u
    error = np.max(np.abs(u - u_ex))

    return error

# Convergence analysis
errors = []
grid_sizes = [10, 20, 40, 80]

for size in grid_sizes:
    error = validate_solver(size, size)
    errors.append(error)

# Plotting the convergence using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=grid_sizes,
    y=errors,
    mode='lines+markers',
    name='Max Error',
    marker=dict(size=8, color='blue'),
    line=dict(width=2)
))

fig.update_layout(
    title='Convergence of the Poisson Solver',
    xaxis_title='Grid Size (N)',
    yaxis_title='Max Error',
    xaxis_type='log',
    yaxis_type='log',
    template='plotly_white',
    showlegend=True
)

fig.show()
computational_time = time.time() - start_time
print(f"Computational time: {computational_time:.2f} seconds")

Computational time: 0.05 seconds
